# Joins

## Prerrequisites

Install Spark and Java in VM

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz

In [ ]:
ls -l # check the .tgz is there

total 267684
drwxr-xr-x 1 root root      4096 Dec  6 14:35 sample_data/
-rw-r--r-- 1 root root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz


In [ ]:
# unzip it
!tar xf spark-3.3.1-bin-hadoop2.tgz

In [ ]:
!pip install -q findspark

In [ ]:
!pip install py4j

# For maps
!pip install folium
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Define the environment

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session 

---

In [ ]:
import findspark
findspark.init("spark-3.3.1-bin-hadoop2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("Joins") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.3.1'

In [ ]:
spark

In [ ]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [ ]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/guitars.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/guitarPlayers.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/bands.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/employees.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/salaries.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/deptmanagers.csv -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/titles.csv -P /dataset

Read JSONs

In [ ]:
guitarsDF = spark.read \
    .option("inferSchema", "true") \
    .json("/dataset/guitars.json")

guitaristsDF = spark.read \
    .option("inferSchema", "true") \
    .json("/dataset/guitarPlayers.json")

bandsDF = spark.read \
    .option("inferSchema", "true") \
    .json("/dataset/bands.json")

In [ ]:
guitarsDF.show(3)
guitaristsDF.show(3)
bandsDF.show(3)


+--------------------+---+------+------------+
|          guitarType| id|  make|       model|
+--------------------+---+------+------------+
|Electric double-n...|  0|Gibson|    EDS-1275|
|            Electric|  5|Fender|Stratocaster|
|            Electric|  1|Gibson|          SG|
+--------------------+---+------+------------+
only showing top 3 rows

+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   0|    [0]|  0|  Jimmy Page|
|   1|    [1]|  1| Angus Young|
|   2| [1, 5]|  2|Eric Clapton|
+----+-------+---+------------+
only showing top 3 rows

+-----------+---+------------+----+
|   hometown| id|        name|year|
+-----------+---+------------+----+
|     Sydney|  1|       AC/DC|1973|
|     London|  0|Led Zeppelin|1968|
|Los Angeles|  3|   Metallica|1981|
+-----------+---+------------+----+
only showing top 3 rows



## Examples

Inner Join =  everything from BOTH DFs for which there is a row in the right DF satisfying the condition


In [ ]:
joinCondition = guitaristsDF.band == bandsDF.id
guitaristsBandsDF = guitaristsDF.join(bandsDF, joinCondition, "inner")
guitaristsBandsDF.show(3)

+----+-------+---+------------+-----------+---+------------+----+
|band|guitars| id|        name|   hometown| id|        name|year|
+----+-------+---+------------+-----------+---+------------+----+
|   1|    [1]|  1| Angus Young|     Sydney|  1|       AC/DC|1973|
|   0|    [0]|  0|  Jimmy Page|     London|  0|Led Zeppelin|1968|
|   3|    [3]|  3|Kirk Hammett|Los Angeles|  3|   Metallica|1981|
+----+-------+---+------------+-----------+---+------------+----+



In [ ]:
# things to bear in mind
# guitaristsBandsDF.select("id", "band").show(3) # will crash because there are two columns named "id"

In [ ]:
# you can rename the column
bandsDFMod = bandsDF.withColumnRenamed("id", "bandId") \
    .withColumnRenamed("name", "bandName")
guitaristsBandsDF2 = guitaristsDF.join(bandsDFMod,
    guitaristsDF.band == bandsDFMod.bandId, "inner").orderBy("name")
guitaristsBandsDF2.show(3)

# there are other options but it is the best practice if it is possible

+----+-------+---+------------+-----------+------+------------+----+
|band|guitars| id|        name|   hometown|bandId|    bandName|year|
+----+-------+---+------------+-----------+------+------------+----+
|   1|    [1]|  1| Angus Young|     Sydney|     1|       AC/DC|1973|
|   0|    [0]|  0|  Jimmy Page|     London|     0|Led Zeppelin|1968|
|   3|    [3]|  3|Kirk Hammett|Los Angeles|     3|   Metallica|1981|
+----+-------+---+------------+-----------+------+------------+----+



Left Outer = everything in the inner join + all the rows in the LEFT DF, with nulls in where the data is missing

In [ ]:
joinCondition2 = guitaristsDF.band == bandsDFMod.bandId
guitaristsDF.join(bandsDFMod, joinCondition2, "left_outer").orderBy("name").show(3)
# see nulls on Eric Calpton

+----+-------+---+------------+--------+------+------------+----+
|band|guitars| id|        name|hometown|bandId|    bandName|year|
+----+-------+---+------------+--------+------+------------+----+
|   1|    [1]|  1| Angus Young|  Sydney|     1|       AC/DC|1973|
|   2| [1, 5]|  2|Eric Clapton|    null|  null|        null|null|
|   0|    [0]|  0|  Jimmy Page|  London|     0|Led Zeppelin|1968|
+----+-------+---+------------+--------+------+------------+----+
only showing top 3 rows



  Right Outer = everything in the inner join + all the rows in the RIGHT DF, with nulls in where the data is missing


In [ ]:
guitaristsDF.join(bandsDFMod, joinCondition2, "right_outer").orderBy("name").show(3)
# see nulls for The Beatles

+----+-------+----+-----------+---------+------+------------+----+
|band|guitars|  id|       name| hometown|bandId|    bandName|year|
+----+-------+----+-----------+---------+------+------------+----+
|null|   null|null|       null|Liverpool|     4| The Beatles|1960|
|   1|    [1]|   1|Angus Young|   Sydney|     1|       AC/DC|1973|
|   0|    [0]|   0| Jimmy Page|   London|     0|Led Zeppelin|1968|
+----+-------+----+-----------+---------+------+------------+----+
only showing top 3 rows



 Outer Join = everything in the inner join + all the rows in BOTH DFs, with nulls in where the data is missing


In [ ]:
guitaristsDF.join(bandsDFMod, joinCondition2, "outer").orderBy("name").show(3)
# see nulls for The Beatles and Eric Clapton

+----+-------+----+------------+---------+------+-----------+----+
|band|guitars|  id|        name| hometown|bandId|   bandName|year|
+----+-------+----+------------+---------+------+-----------+----+
|null|   null|null|        null|Liverpool|     4|The Beatles|1960|
|   1|    [1]|   1| Angus Young|   Sydney|     1|      AC/DC|1973|
|   2| [1, 5]|   2|Eric Clapton|     null|  null|       null|null|
+----+-------+----+------------+---------+------+-----------+----+
only showing top 3 rows



  Semi-joins = everything in the LEFT DF for which there is a row in the right DF satisfying the condition


In [ ]:
guitaristsDF.join(bandsDFMod, joinCondition2, "left_semi").orderBy("name").show(3)


+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   1|    [1]|  1| Angus Young|
|   0|    [0]|  0|  Jimmy Page|
|   3|    [3]|  3|Kirk Hammett|
+----+-------+---+------------+



Anti-joins = everything in the LEFT DF for which there is NO row in the right DF satisfying the condition


In [ ]:
guitaristsDF.join(bandsDFMod, joinCondition2, "anti").orderBy("name").show(3)

+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   2| [1, 5]|  2|Eric Clapton|
+----+-------+---+------------+



Cross-join = takes every instance in the LEFT DF and create a new one for every intance in the right DF (not very common)

In [ ]:
guitaristsDF.crossJoin(bandsDFMod).show(20)

+----+-------+---+------------+-----------+------+------------+----+
|band|guitars| id|        name|   hometown|bandId|    bandName|year|
+----+-------+---+------------+-----------+------+------------+----+
|   0|    [0]|  0|  Jimmy Page|     Sydney|     1|       AC/DC|1973|
|   1|    [1]|  1| Angus Young|     Sydney|     1|       AC/DC|1973|
|   2| [1, 5]|  2|Eric Clapton|     Sydney|     1|       AC/DC|1973|
|   3|    [3]|  3|Kirk Hammett|     Sydney|     1|       AC/DC|1973|
|   0|    [0]|  0|  Jimmy Page|     London|     0|Led Zeppelin|1968|
|   1|    [1]|  1| Angus Young|     London|     0|Led Zeppelin|1968|
|   2| [1, 5]|  2|Eric Clapton|     London|     0|Led Zeppelin|1968|
|   3|    [3]|  3|Kirk Hammett|     London|     0|Led Zeppelin|1968|
|   0|    [0]|  0|  Jimmy Page|Los Angeles|     3|   Metallica|1981|
|   1|    [1]|  1| Angus Young|Los Angeles|     3|   Metallica|1981|
|   2| [1, 5]|  2|Eric Clapton|Los Angeles|     3|   Metallica|1981|
|   3|    [3]|  3|Kirk Hammett|Los

*Union Join is just an Union (note it only works if both DFs have the same schema)

## Exercises
1. Read employees.csv, deptmanagers.csv, salaries.csv and titles.csv to a DFs.
2. Show all employees and their max salary (there can be different salaries registered for the same employee)
3. Show all employees who were never managers. Check it by getting all the managers and checking that they are not in the table
4. Find the job titles of the best paid 10 employees in the company (note that there can be different titles registered for the same employee)

Exercise 1

In [ ]:
employeesDF = spark.read.option("header", "true").csv("/dataset/employees.csv")
salariesDF = spark.read.option("header", "true").csv("/dataset/salaries.csv")
deptManagersDF = spark.read.option("header", "true").csv("/dataset/deptmanagers.csv")
titlesDF = spark.read.option("header", "true").csv("/dataset/titles.csv")

In [ ]:
employeesDF.show(3)

+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24|
| 10020|1952-12-24|    Mayuko|  Warwick|     M|1991-01-26|
| 10030|1958-07-14|     Elvis|  Demeyer|     M|1994-02-17|
+------+----------+----------+---------+------+----------+
only showing top 3 rows



In [ ]:
employeesDF.count()

30003

Exercise 2

In [ ]:
salariesDF.filter(col("emp_no") == 100000).show(4)

+------+------+----------+----------+
|emp_no|salary| from_date|   to_date|
+------+------+----------+----------+
|100000| 41408|1991-07-02|1992-07-01|
|100000| 43119|1992-07-01|1993-07-01|
|100000| 46991|1993-07-01|1994-07-01|
|100000| 48751|1994-07-01|1995-07-01|
+------+------+----------+----------+
only showing top 4 rows



In [ ]:
maxSalariesPerEmpNoDF = salariesDF.groupBy("emp_no").agg(max("salary").cast("long").alias("maxSalary"))
maxSalariesPerEmpNoDF.filter(col("emp_no") == 100000).show(4)

+------+---------+
|emp_no|maxSalary|
+------+---------+
|100000|    64961|
+------+---------+



In [ ]:
employeesSalariesDF = employeesDF.join(maxSalariesPerEmpNoDF, 
    employeesDF.emp_no == maxSalariesPerEmpNoDF.emp_no, "inner")
employeesSalariesDF.show(3)

+------+----------+----------+---------+------+----------+------+---------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|emp_no|maxSalary|
+------+----------+----------+---------+------+----------+------+---------+
|100000|1956-01-11|  Hiroyasu|    Emden|     M|1991-07-02|100000|    64961|
|100010|1959-11-06|    Youpyo|   Aamodt|     M|1991-10-04|100010|    71957|
|100020|1956-05-20|    Pramod|     Pelc|     M|1987-03-01|100020|    70286|
+------+----------+----------+---------+------+----------+------+---------+
only showing top 3 rows



In [ ]:
# note it is equivalent to
employeesSalariesDF2 = employeesDF.join(maxSalariesPerEmpNoDF, "emp_no")
employeesSalariesDF2.show(3)

+------+----------+----------+---------+------+----------+---------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|maxSalary|
+------+----------+----------+---------+------+----------+---------+
|100000|1956-01-11|  Hiroyasu|    Emden|     M|1991-07-02|    64961|
|100010|1959-11-06|    Youpyo|   Aamodt|     M|1991-10-04|    71957|
|100020|1956-05-20|    Pramod|     Pelc|     M|1987-03-01|    70286|
+------+----------+----------+---------+------+----------+---------+
only showing top 3 rows



Exercise 3

In [ ]:
empNeverManagersDF = employeesDF.join(
    deptManagersDF,
    employeesDF.emp_no == deptManagersDF.emp_no,
    "left_anti")

empNeverManagersDF.show(3)

+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24|
| 10020|1952-12-24|    Mayuko|  Warwick|     M|1991-01-26|
| 10030|1958-07-14|     Elvis|  Demeyer|     M|1994-02-17|
+------+----------+----------+---------+------+----------+
only showing top 3 rows



In [ ]:
# alternatively
empNeverManagersDF2 = employeesDF.join(
    deptManagersDF,
    employeesDF.emp_no != deptManagersDF.emp_no,
    "left_semi")

empNeverManagersDF2.show(3)

+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24|
| 10020|1952-12-24|    Mayuko|  Warwick|     M|1991-01-26|
| 10030|1958-07-14|     Elvis|  Demeyer|     M|1994-02-17|
+------+----------+----------+---------+------+----------+
only showing top 3 rows



In [ ]:
# managers
managersDF = employeesDF.join(
    deptManagersDF,
    employeesDF.emp_no == deptManagersDF.emp_no,
    "inner")

managersDF.show(6)

+------+----------+----------+---------+------+----------+-------+------+----------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|dept_no|emp_no| from_date|   to_date|
+------+----------+----------+---------+------+----------+-------+------+----------+----------+
|110420|1963-07-27|     Oscar| Ghazalie|     M|1992-02-05|   d004|110420|1996-08-30|9999-01-01|
|110800|1963-02-07|    Sanjoy|  Quadeer|     F|1986-08-12|   d006|110800|1991-09-12|1994-06-28|
|111400|1959-11-09|      Arie|  Staelin|     M|1985-01-01|   d008|111400|1985-01-01|1991-04-08|
+------+----------+----------+---------+------+----------+-------+------+----------+----------+



In [ ]:
empNeverManagersDF.filter((col("first_name") == "Oscar") & (col("emp_no") == 110420)).show(3)

+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
+------+----------+----------+---------+------+----------+



In [ ]:
# it is fine

Exercise 4

In [ ]:
titlesDF.filter(col("emp_no") == 10040).show(4)
# windowing will be useful for this cases

+------+---------------+----------+----------+
|emp_no|          title| from_date|   to_date|
+------+---------------+----------+----------+
| 10040|       Engineer|1993-02-14|1999-02-14|
| 10040|Senior Engineer|1999-02-14|9999-01-01|
+------+---------------+----------+----------+



In [ ]:
mostRecentJobTitlesDateDF = titlesDF.groupBy("emp_no").agg(max("to_date").alias("latest_date"))
mostRecentJobTitlesDateDF.filter(col("emp_no") == 10040).show(4)

+------+-----------+
|emp_no|latest_date|
+------+-----------+
| 10040| 9999-01-01|
+------+-----------+



In [ ]:
mostRecentJobTitlesDF = titlesDF.join(mostRecentJobTitlesDateDF, (titlesDF.emp_no == mostRecentJobTitlesDateDF.emp_no) 
    & (titlesDF.to_date == mostRecentJobTitlesDateDF.latest_date), "semi").drop("from_date")
mostRecentJobTitlesDF.show(6)

+------+---------------+----------+
|emp_no|          title|   to_date|
+------+---------------+----------+
| 10010|       Engineer|9999-01-01|
| 10020|       Engineer|9999-01-01|
| 10030|Senior Engineer|9999-01-01|
| 10040|Senior Engineer|9999-01-01|
| 10050|   Senior Staff|9999-01-01|
| 10060|   Senior Staff|9999-01-01|
+------+---------------+----------+
only showing top 6 rows



In [ ]:
bestPaidEmployeesDF = employeesSalariesDF.orderBy(col("maxSalary").desc()).limit(10)
bestPaidEmployeesDF.show()

+------+----------+----------+---------+------+----------+------+---------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|emp_no|maxSalary|
+------+----------+----------+---------+------+----------+------+---------+
|205000|1956-01-14|  Charmane| Griswold|     M|1990-06-23|205000|   153715|
|246120|1961-05-26|      Arnd|    Junot|     F|1985-04-07|246120|   146292|
|257360|1955-10-30|   Honglan|  Otillio|     F|1985-12-09|257360|   144748|
|107140|1960-05-05|    Ortrud|Diderrich|     M|1987-12-08|107140|   142506|
|282030|1963-09-24|     Sergi|   Leslie|     F|1989-10-09|282030|   142184|
|282370|1964-05-29|    Cedric|    Rossi|     F|1987-06-06|282370|   141488|
|447240|1953-12-25|      Maja|  Riefers|     M|1987-11-06|447240|   140332|
|295800|1952-07-14|  Jingling|  Weisert|     F|1986-08-17|295800|   139973|
|296250|1961-04-30|    Krisda| Papsdorf|     M|1989-09-21|296250|   138716|
|411990|1955-02-23|     Luigi|   Renear|     M|1987-05-02|411990|   138273|
+------+----

In [ ]:
bestPaidJobsDF = bestPaidEmployeesDF.join(mostRecentJobTitlesDF, "emp_no")
bestPaidJobsDF.show()

+------+----------+----------+---------+------+----------+------+---------+------------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|emp_no|maxSalary|       title|   to_date|
+------+----------+----------+---------+------+----------+------+---------+------------+----------+
|107140|1960-05-05|    Ortrud|Diderrich|     M|1987-12-08|107140|   142506|Senior Staff|9999-01-01|
|205000|1956-01-14|  Charmane| Griswold|     M|1990-06-23|205000|   153715|Senior Staff|9999-01-01|
|246120|1961-05-26|      Arnd|    Junot|     F|1985-04-07|246120|   146292|Senior Staff|9999-01-01|
|257360|1955-10-30|   Honglan|  Otillio|     F|1985-12-09|257360|   144748|Senior Staff|9999-01-01|
|282030|1963-09-24|     Sergi|   Leslie|     F|1989-10-09|282030|   142184|Senior Staff|9999-01-01|
|282370|1964-05-29|    Cedric|    Rossi|     F|1987-06-06|282370|   141488|Senior Staff|9999-01-01|
|295800|1952-07-14|  Jingling|  Weisert|     F|1986-08-17|295800|   139973|Senior Staff|9999-01-01|
